### Prepare Dataset of 5 Image

In [ ]:
from PIL import Image, ImageOps

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import cv2
import numpy as np

In [ ]:
def get_edited_folders():
  return ['bordered/#000', 'bordered/#fff', 'cropped/center', 'cropped/left_half', 'cropped/lower_half', 'cropped/lowerleft_quarter', 'cropped/lowerright_quarter', 'cropped/right_half', 'cropped/upper_half', 'cropped/upperleft_quarter', 'cropped/upperright_quarter', 'filtered/aden', 'filtered/inkwell', 'filtered/lofi', 'mirrored', 'rotated/45', 'rotated/90', 'rotated/180']

In [ ]:
# Process Data

image_list = ["BwjvPEAgrRr.jpg", "BxNDSVPAXRr.jpg", "BxPSFbeHxrR.jpg", "BtiVFeUgrrR.jpg"]

labels = []

l1 = "Dog"
l2 = "Kid"
l3 = "Party"
l4 = "Skeeing"

for _ in range(19):
    labels.append(l1)
    labels.append(l2)
    labels.append(l3)
    labels.append(l4)

# Images

image_paths = []

original_basepath = '/content/drive/My Drive/DL project/data_plain/r/r'

# Original Images
for img in image_list:
  path1 = original_basepath+"/"+img
  image_paths.append(path1)

edited_folders = get_edited_folders()
edited_basepath = original_basepath.replace('data_plain', 'data_prepared')

# Edited images
for folder in edited_folders:
  edited_folders = get_edited_folders()
  for img in image_list:
    path2 = f'{edited_basepath}/{folder}/{img}'
    image_paths.append(path2)

In [ ]:
print(labels)

print(image_paths)

['Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing', 'Dog', 'Kid', 'Party', 'Skeeing']
['/content/drive/My Drive/DL project/data_plain/r/r/BwjvPEAgrRr.jpg', '/content/drive/My Drive/DL project/data_plain/r/r/BxNDSVPAXRr.jpg', '/content/drive/My Drive/DL project/data_plain/r/r/BxPSFbeHxrR.jpg', '/content/drive/My Drive/DL project/data_plain/r/r/BtiVFeUgrrR.jpg', '/content/drive/My Drive/DL project/data_prepared/r/r/bordered/#000/BwjvPEAg

### Extracting ResNet-50 embeddings

In [ ]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from PIL import Image
import numpy as np
import os
import torch

def get_edited_folders():
  return ['bordered/#000', 'bordered/#fff', 'cropped/center', 'cropped/left_half', 'cropped/lower_half', 'cropped/lowerleft_quarter', 'cropped/lowerright_quarter', 'cropped/right_half', 'cropped/upper_half', 'cropped/upperleft_quarter', 'cropped/upperright_quarter', 'filtered/aden', 'filtered/inkwell', 'filtered/lofi', 'mirrored', 'rotated/45', 'rotated/90', 'rotated/180']


image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

def get_em(filepath):
  image = Image.open(filepath)
  inputs = image_processor(image, return_tensors="pt")
  with torch.no_grad():
      logits = model(**inputs).logits
  return logits

resnet_embeddings = []
for path in image_paths:
  resnet_embeddings.append(get_em(path))

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/103M [00:00<?, ?B/s]

In [ ]:
len(resnet_embeddings)

76

### T-SNE plot for resnet embeddings

In [ ]:
# Concatenate tensors into a single array
concatenated_array = np.concatenate([tensor.numpy().flatten() for tensor in resnet_embeddings], axis=0)

# Reshape the concatenated array to have shape (76, 1000)
reshaped_array = concatenated_array.reshape(len(resnet_embeddings), -1)

In [ ]:
from sklearn.manifold import TSNE
import time

start = time.time()
tsne = TSNE(n_components = 3, random_state=0)
projections = tsne.fit_transform(reshaped_array)
end = time.time()
print(f"generating projections with T-SNE took: {(end-start):.2f} sec")

generating projections with T-SNE took: 0.80 sec


In [ ]:
len(projections)

76

In [ ]:
import plotly.express as px
fig = px.scatter_3d(
    projections, x=0, y=1, z=2,
    color=labels, hover_data=[image_paths]
)
fig.update_traces(marker_size=8)
fig.show()

### UMAP for resnet embeddings

In [ ]:
! pip install --upgrade umap-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
import umap.umap_ as umap
import time

In [ ]:
start = time.time()
projections_umap = umap.UMAP(n_components=3).fit_transform(reshaped_array)
end = time.time()
print(f"generating projections with UMAP took: {(end-start):.2f} sec")

generating projections with UMAP took: 10.08 sec


In [ ]:
fig = px.scatter_3d(
    projections_umap, x=0, y=1, z=2,
    color=labels, hover_data=[image_paths]
)
fig.update_traces(marker_size=8)
fig.show()

### Using cosine similarity to detect duplicates

In [ ]:
import numpy as np

def normalize(vec: np.ndarray):
    return vec / np.linalg.norm(vec, axis=1, keepdims=True)

In [ ]:
normalized_embeddings = normalize(reshaped_array)
cosine_sim_matrix = np.dot(normalized_embeddings, normalized_embeddings.T)

In [ ]:
import numpy as np
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(z=cosine_sim_matrix, colorscale='Viridis'))

fig.update_layout(
    title="Cosine Similarity Matrix",
    xaxis_title="Image Index",
    yaxis_title="Image Index",
    font=dict(size=14),
    width=1000,
    height=1000,
    margin=dict(t=100, r=100, b=100, l=100),
)

fig.show()